## <font color=darkblue>Implémentation du maximum de vraisemblance pour le modèle de régression logistique</font>

L'objectif de la régression logistique est d'expliquer une variable binaire par des observations réelles indépendantes. Pour y parvenir on estime le paramètre inconnu qui lie les observations et la variable binaire. On se place donc dans un cadre paramétrique.

# <font color=darkgreen>1. Le modèle</font>

**Définition** Le `modèle de classification supervisée` correspond à la donnée de $\mathcal{D}_n = (X_i,Y_i)_{1\leq i\leq n}$ les observations où $(X_i,Y_i) \stackrel{iid}{\sim} (X,Y)$ un couple générique de loi $P_{X,Y}$ où $X \in \mathbb{R}^d$, est la variable dite explicative (de dimension $d \in \mathbb{N}$) et $Y \in \{ 0,1 \}$ la variable expliquée.

**Définition** On appelle fonction de régression la fonction
$$
    \eta : x \in \mathbb{R}^d \mapsto \eta(x) = \mathbb{E}[Y|X=x] = P(Y=1|X=x)
$$

**Définition** Le `modèle de régression logistique` correspond à un modèle de classification supervisée sur lequel on fait l'`hypothèse logistique` : $\exists \beta \in \mathbb{R}^d, \exists a \in \mathbb{R}, \forall x \in \mathbb{R}^d$,
$$
    log\left( \frac{\eta(x)}{1-\eta(x)} \right) = \beta^Tx+a
$$

Il est important de remarquer que $1 - \eta(x) = P(Y=0|X=x)$ donc
$ P(Y=1|X=x) \geq P(Y=0|X=x) \iff log\left( \frac{\eta(x)}{1-\eta(x)} \right) \geq 0 $

Puisque $\eta(x) = \frac{exp(\beta^Tx+a)}{1+exp\beta^Tx+a)}$, on a $\mathcal{L}((Y_i)_{1\leq i\leq n} | (X_i)_{1\leq i\leq n}) = \bigotimes_{i=1}^n \mathcal{B}(\eta(X_i)) = \bigotimes_{i=1}^n \mathcal{B}\left(\frac{exp(\beta^TX_i+a)}{1+exp\beta^TX_i+a)}\right)$

>**Rappel** : si les $(X_i)_{1\leq i\leq n}$ sont indépendants à valeurs dans $(X,\mathcal{X})$ muni d'une mesure de référence $\mu$, et que la loi de $X_1$ a une densité $p_{\theta_0}$ par rapport à $\mu$ où $p_{\theta_0} \in \left\{ p_{\theta}, \theta \in \Theta \right\}$, par indépendance des $X_i$, la fonction de `vraisemblance` s'écrit alors
$$L_n : \theta \mapsto p_{\theta}(X_1,...,X_n) = \prod_{i=1}^{n}p_{\theta}(X_i)
$$
Et la `log-vraisemblance`
$$
l_n : \theta \mapsto log(p_{\theta}(X_1,...,X_n)) = \sum_{i=1}^{n}log(p_{\theta}(X_i))
$$
On appelle `maximum de vraisemlance` tout $\hat{\theta}_n^{MV}$ tel que
$$
    \hat{\theta}_n^{MV} \in \underset{\theta \in \Theta}{\mathrm{argmin}}\:l_n(\theta)
$$

Finalement on obtient la log-vraisemblance du modèle logistique
<font color=darkred>$$
    l_n(\beta,a) = \sum_{i=1}^n \left( Y_i\left(\beta^TX_i+a\right) - log\left(1+e^{\beta^TX_i+a}\right) \right)
$$</font>

>**Remarque** : L'estimateur du maximum de vraisemblance est un cas particulier d'estimateurs plus généraux : les M-estimateurs.\
Un *M-estimateur* est ...

# 2. Maximisation de la vraisemblance

## a) Méthode du 1er ordre : montée de gradient

## b) Méthode du 2nd ordre : Newton-Raphson

## c) Expectation-Maximisation (EM)

# Références

<a href="https://en.wikipedia.org/wiki/Logistic_regression" title="WpLR">Wikipedia - Logistic regression</a>